In [1]:
import json

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
with open("hmao_npa.txt") as file:
    text = file.read()

raw_docs = text.split("\n")
index = [i for i in range(len(raw_docs)) if i % 2 == 0]

docs_df = (
    pd.DataFrame({"index": range(len(raw_docs)), "document": raw_docs})
    .iloc[index]
    .reset_index()
    .drop(columns=["index", "level_0"])
)
    
docs_df

,document
0,ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВ...
1,ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬСТВА ХАНТЫ-МАНСИЙСКОГО ...
2,ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВ...
3,ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬСТВА ХАНТЫ-МАНСИЙСКОГО ...
4,ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬСТВА ХАНТЫ-МАНСИЙСКОГО ...
...,...
1852,ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВ...
1853,ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГ...
1854,ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬСТВА ХАНТЫ-МАНСИЙСКОГО ...
1855,ПОСТАНОВЛЕНИЕ ПРАВИТЕЛЬСТВА ХАНТЫ-МАНСИЙСКОГО ...


In [3]:
qa_df = pd.read_excel("v2_ragas_npa_dataset_firstPart.xlsx")
qa_df['contexts'] = qa_df['contexts'].apply(eval)
qa_df['metadata'] = qa_df['metadata'].apply(eval)
qa_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"Каков объем экспорта услуг категории ""Поездки""...","[Увеличение объема экспорта услуг категории ""П...","Объем экспорта услуг категории ""Поездки"" в рег...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
1,"Каковы были основные мероприятия, реализуемые ...",[и | |развития | | |Ханты-Мансийского |исполни...,"Основные мероприятия, реализуемые Правительств...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
2,Каков размер финансирования из федерального бю...,[не относящиеся к районам Крайнего Севера и пр...,Ответ на данный вопрос отсутствует в контексте.,simple,[{'file_path': '/home/cias/projects/other/raga...,True
3,Какой комплексный центр социального обслуживан...,"[|2.2. |Центр социальной |240 к/мест |7580,0 |...",Комплексный центр социального обслуживания нас...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
4,Какое значение имеет экологическое образование...,[(полного)общего образования и начального проф...,Экологическое образование играет важную роль в...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
5,Как регулирование трудовой миграции влияет на ...,"[|2.4. Проведение |ДФГСЗН (по |0,0 тыс. |44 |4...",Регулирование трудовой миграции оказывает знач...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
6,"Каковы основные должности, соответствующие каж...",[| ||————————————————————|————————————————————...,"Основные должности, соответствующие каждому кв...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
7,Какие документы необходимы для предоставления ...,"[на основании договора); 4) документ, подтверж...",Для предоставления муниципальной услуги по пол...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
8,Какова общая сумма по городу Когалыму за 2013-...,"[41502,96 4421289657,71 756296153,79 313690263...",Общая сумма по городу Когалыму за 2013-2014 го...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
9,Каким образом федеральное законодательство о к...,[исходя из приоритетов социально-экономическог...,Федеральное законодательство о концессионных с...,simple,[{'file_path': '/home/cias/projects/other/raga...,True


In [4]:
from langchain_core.documents import Document

documents = docs_df['document'].apply(Document).tolist()
len(documents)

1857

In [8]:
rom langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=256,
    length_function=len,
    is_separator_regex=False
)

chunks = text_splitter.split_documents(documents)
len(chunks)

15774

In [14]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=chunks, embedding=local_embeddings)

In [15]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1")

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input_: format_docs(input_["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

def qa(num):
    question = qa_df['question'].iloc[num]
    print("Question:", question)
    print()
    docs = vectorstore.similarity_search(question, k=10)
    
    rag_output = chain.invoke({"context": docs, "question": question})
    print("Rag output:", rag_output)
    print()
    
    gt = qa_df['ground_truth'].iloc[num]
    print("Ground truth:", gt)
    print()


def get_answer(question):
    docs = vectorstore.similarity_search(question, k=10)
    
    rag_output = chain.invoke({"context": docs, "question": question})
    return rag_output

qa(2)

Question: Каков размер финансирования из федерального бюджета на реализацию мероприятий государственной программы "Сотрудничество" в 2013 году?

Rag output: Размер финансирования из федерального бюджета на реализацию мероприятий государственной программы "Сотрудничество" в 2013 году составляет 0,05% от размера федерального бюджета.

Ground truth: Ответ на данный вопрос отсутствует в контексте.



In [19]:
from tqdm import tqdm

# считается примерно час
answers = []
for i in tqdm(range(qa_df.shape[0])):
    q = qa_df['question'].iloc[i]
    
    answers.append(get_answer(q))

len(answers)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [1:31:46<00:00, 11.01s/it]


500

In [20]:
qa_df['rag_answers'] = answers

In [17]:
from datasets import Dataset 
from ragas.metrics import faithfulness, answer_correctness, context_precision, context_utilization, context_recall
from ragas import evaluate


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times'],
    'contexts' : [
        ['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
        ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']
    ],
}


dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset,
    llm=model,
    metrics=[faithfulness, answer_correctness, context_precision, context_utilization, context_recall],
)
score.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[1]: TimeoutError()


,question,answer,ground_truth,contexts,faithfulness,answer_correctness,context_precision,context_utilization,context_recall
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",[The First AFL–NFL World Championship Game was...,NaN,NaN,1.0,1.0,1.0
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,"[The Green Bay Packers...Green Bay, Wisconsin....",NaN,NaN,0.0,0.0,0.0


In [18]:
# Считается 7:37

from datasets import Dataset 
from ragas.metrics import faithfulness, answer_correctness, context_precision, context_utilization, context_recall
from ragas import evaluate


data_samples = {
    'question': qa_df['question'].tolist(),
    'answer': qa_df['rag_answers'].tolist(),
    'ground_truth': qa_df['ground_truth'].tolist(),
    'contexts' : qa_df['contexts'].tolist()
}


dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset,
    llm=model,
    metrics=[faithfulness, answer_correctness, context_precision, context_utilization, context_recall],
)
pd_score = score.to_pandas()
pd_score

KeyError: 'rag_answers'